<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Exploratory Data Analysis with SQL
Estimated time needed: 60 minutes


# Introduction
Using this R notebook you will perform exploratory data analysis using SQL queries with the RSQLite R package. You will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your SQL Lite connection

#### Install RSQLite package
The RSQlite  package needs to be installed in your notebook. Let’s load the RSQLite package by clicking on the following cell and executing it (Shift+Enter):


In [ ]:
if (!require("RSQLite", quietly = TRUE)) install.packages("RSQLite")

#### Restart Kernel

After installing the RSQLite package, it is necessary to restart R Kernel. Click **Kernel** > **Restart Kernel** from the main menu. This will register the newly installed packages. Now proceed to load the RSQLite package. 


#### Load RSQLite
Load the 'RSQLite' library, and use the 'dbConnect( )' function as you did in the previous labs to establish the connection to your SQLite database.  
You are now ready to start running SQL queries using the RSQLite library as you did in Course 3.


In [ ]:
library(RSQLite)
library(readr)
library(dplyr)

In [ ]:
# Connect to SQLite database in project output directory
db_path <- file.path("C:/Users/Diaa/data.science.with.r/project5-capstone/output", "eda_capstone.sqlite")
conn <- dbConnect(SQLite(), dbname = db_path)
conn

Download the following csv files:

*   [WORLD_CITIES](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/world_cities.csv)

*   [BIKE_SHARING_SYSTEMS](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/bike_sharing_systems.csv)

*   [CITIES_WEATHER_FORECAST](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/cities_weather_forecast.csv)

*   [SEOUL_BIKE_SHARING](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/seoul_bike_sharing.csv)

and load the csv's  into 4 tables as mentioned below
* SEOUL_BIKE_SHARING

* CITIES_WEATHER_FORECAST

* BIKE_SHARING_SYSTEMS 

*  WORLD_CITIES

> Hint : Use the read_csv() function and dbWriteTable() functions


In [ ]:
# Load local datasets and write to SQLite tables
seoul_path <- "C:/Users/Diaa/data.science.with.r/project5-capstone/data/raw_seoul_bike_sharing.csv"
weather_path <- "C:/Users/Diaa/data.science.with.r/project5-capstone/output/cities_weather_forecast_cleaned.csv"
bike_path <- "C:/Users/Diaa/data.science.with.r/project5-capstone/output/bike_sharing_cleaned.csv"
world_path <- "C:/Users/Diaa/data.science.with.r/project5-capstone/output/worldcities_cleaned.csv"

# Ensure DATE is character for the raw Seoul dataset
seoul <- read_csv(seoul_path, col_types = cols(Date = col_character()), show_col_types = FALSE)
names(seoul) <- toupper(names(seoul))
dbWriteTable(conn, "SEOUL_BIKE_SHARING", seoul, overwrite = TRUE)

weather <- read_csv(weather_path, show_col_types = FALSE)
dbWriteTable(conn, "CITIES_WEATHER_FORECAST", weather, overwrite = TRUE)

bike <- read_csv(bike_path, show_col_types = FALSE)
if ("CITY" %in% names(bike)) bike$CITY <- gsub("\\[[^\\]]+\\]", "", bike$CITY)
if ("SYSTEM" %in% names(bike)) bike$SYSTEM <- gsub("\\[[^\\]]+\\]", "", bike$SYSTEM)
if ("BICYCLES" %in% names(bike)) bike$BICYCLES <- suppressWarnings(as.numeric(bike$BICYCLES))
dbWriteTable(conn, "BIKE_SHARING_SYSTEMS", bike, overwrite = TRUE)

world <- read_csv(world_path, show_col_types = FALSE)
dbWriteTable(conn, "WORLD_CITIES", world, overwrite = TRUE)


## Task 1 - Record Count
#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [ ]:
dbGetQuery(conn, "SELECT COUNT(*) AS row_count FROM SEOUL_BIKE_SHARING")

## Task 2 - Operational Hours
#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [ ]:
dbGetQuery(conn, "SELECT COUNT(*) AS hours_non_zero FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT > 0")

## Task 3 - Weather Outlook
#### Query the the weather forecast for Seoul over the next 3 hours.
Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [ ]:
dbGetQuery(conn, "SELECT CITY, FORECAST_TIME, WEATHER, TEMP, HUMIDITY\n FROM CITIES_WEATHER_FORECAST\n WHERE CITY = 'Seoul'\n ORDER BY FORECAST_TIME ASC\n LIMIT 1")

## Task 4 - Seasons
#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [ ]:
dbGetQuery(conn, "SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING ORDER BY SEASONS")

## Task 5 - Date Range
#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [ ]:
dbGetQuery(conn, "SELECT\n MIN(substr(DATE,7,4)||'-'||substr(DATE,4,2)||'-'||substr(DATE,1,2)) AS first_date_iso,\n MAX(substr(DATE,7,4)||'-'||substr(DATE,4,2)||'-'||substr(DATE,1,2)) AS last_date_iso\n FROM SEOUL_BIKE_SHARING")

## Task 6 - Subquery - 'all-time high'
#### determine which date and hour had the most bike rentals.


### Solution 6


In [ ]:
dbGetQuery(conn, "SELECT DATE, HOUR, RENTED_BIKE_COUNT\n FROM SEOUL_BIKE_SHARING\n WHERE RENTED_BIKE_COUNT = (SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING)")

## Task 7 - Hourly popularity and temperature by season
#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [ ]:
dbGetQuery(conn, "SELECT SEASONS, HOUR,\n AVG(TEMPERATURE) AS avg_temperature,\n AVG(RENTED_BIKE_COUNT) AS avg_bike_count\n FROM SEOUL_BIKE_SHARING\n GROUP BY SEASONS, HOUR\n ORDER BY avg_bike_count DESC\n LIMIT 10")

## Task 8 - Rental Seasonality
#### Find the average hourly bike count during each season. 
Also include the minimum, maximum, and standard deviation of the hourly bike count for each season. 

> Hint : Use the SQRT(AVG(col*col) - AVG(col)*AVG(col) ) function where col refers to your column name for finding the standard deviation


### Solution 8


In [ ]:
dbGetQuery(conn, "SELECT SEASONS,\n AVG(RENTED_BIKE_COUNT) AS avg_bike,\n MIN(RENTED_BIKE_COUNT) AS min_bike,\n MAX(RENTED_BIKE_COUNT) AS max_bike,\n SQRT(AVG(RENTED_BIKE_COUNT*RENTED_BIKE_COUNT) - AVG(RENTED_BIKE_COUNT)*AVG(RENTED_BIKE_COUNT)) AS std_bike\n FROM SEOUL_BIKE_SHARING\n GROUP BY SEASONS\n ORDER BY avg_bike DESC")

Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality
#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?
Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all. 


### Solution 9


In [ ]:
dbGetQuery(conn, "SELECT SEASONS,\n AVG(TEMPERATURE) AS avg_temp,\n AVG(HUMIDITY) AS avg_humidity,\n AVG(WIND_SPEED) AS avg_wind_speed,\n AVG(VISIBILITY) AS avg_visibility,\n AVG(DEW_POINT_TEMPERATURE) AS avg_dew_point,\n AVG(SOLAR_RADIATION) AS avg_solar_radiation,\n AVG(RAINFALL) AS avg_rainfall,\n AVG(SNOWFALL) AS avg_snowfall,\n AVG(RENTED_BIKE_COUNT) AS avg_bike\n FROM SEOUL_BIKE_SHARING\n GROUP BY SEASONS\n ORDER BY avg_bike DESC")

## Task 10 - Total Bike Count and City Info for Seoul
#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.
Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [ ]:
dbGetQuery(conn, "SELECT w.CITY, w.COUNTRY, w.LAT, w.LNG, w.POPULATION,\n SUM(b.BICYCLES) AS TOTAL_BICYCLES\n FROM WORLD_CITIES w, BIKE_SHARING_SYSTEMS b\n WHERE b.CITY LIKE 'Seoul%' AND w.CITY = 'Seoul'\n GROUP BY w.CITY, w.COUNTRY, w.LAT, w.LNG, w.POPULATION")

## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system
#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.
Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [ ]:
dbGetQuery(conn, "SELECT w.CITY, w.COUNTRY, w.LAT, w.LNG, w.POPULATION, t.TOTAL_BICYCLES\n FROM (SELECT CITY, SUM(COALESCE(BICYCLES,0)) AS TOTAL_BICYCLES\n       FROM BIKE_SHARING_SYSTEMS\n       GROUP BY CITY) t\n JOIN WORLD_CITIES w ON w.CITY = t.CITY\n WHERE t.TOTAL_BICYCLES BETWEEN 15000 AND 20000\n ORDER BY t.TOTAL_BICYCLES DESC")

In [ ]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>
<h4> Lakshmi Holla </h4>


## Other Contributor(s)

<h4>  Malika Singla </h4>

<!--## Change log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2022-03-03        | 1.0     |Lakshmi Holla  | Created lab using sqlite3          |-->

## <h3 align="center"> © IBM Corporation 2022. All rights reserved. <h3/>
